In [1]:
import tweepy
import pandas as pd
import ast
import nltk
from nltk.corpus import brown
from nltk import word_tokenize
from nltk.probability import FreqDist
import math
import warnings;
warnings.filterwarnings('ignore');

In [2]:
#Preprocessing
# read date
df = pd.read_csv('tweets_message_tokyo.csv', encoding = "utf-8")
# drop rows which has at least one element to be Nan(drop 63562 here)
df = df.dropna()
#convert text to utf8
df['text'] = df['text'].apply(lambda x: ast.literal_eval(x).decode('utf8'))
# #sort df based on time
# df_sort = df.sort_values(by=['time'])
# df_sort

In [3]:
from pathlib import Path 
#save data in dictionary(including dates from '2021-7-19' to '2021-8-11')
date_range = pd.date_range(start='2021-07-19', end='2021-08-11')

## Read CSV files back to dataframe dict
df_dict = {}
for i in date_range:
    date = i.strftime('%Y-%m-%d')
    filepath = Path('dataframes_by_date/'+date+".csv")
    df_dict[date] = pd.read_csv(filepath, encoding = "utf-8")
df_dict["2021-07-31"]

,Unnamed: 0,id,time,text,city_name,coordinates,place
0,39424,1421259330838540295,2021-07-31 00:00:04+00:00,#TeamGB won #triathlon #Olympics2020 #Toyko2020,"West Chester, OH","[[[-84.4832977, 39.2957423], [-84.346523, 39.2...",Place(_api=<tweepy.api.API object at 0x2b23538...
1,17157,1421259710884417537,2021-07-31 00:01:35+00:00,Opposite ends of the athletic body type @laury...,"Johnson City, TN","[[[-82.4759055, 36.2592231], [-82.300523, 36.2...",Place(_api=<tweepy.api.API object at 0x2b23538...
2,39420,1421259737694445583,2021-07-31 00:01:41+00:00,@AnglingNArchery @USABaseball @TeamUSA @WBSC @...,"Zanesville, OH","[[[-82.121534, 39.908312], [-81.976901, 39.908...",Place(_api=<tweepy.api.API object at 0x2b23538...
3,30184,1421259768891576321,2021-07-31 00:01:49+00:00,It’s the Olympics. There are no weekends. See ...,"New Canaan, CT","[[[-73.5537, 41.11369], [-73.4489288, 41.11369...",Place(_api=<tweepy.api.API object at 0x2b23538...
4,54236,1421259812097171461,2021-07-31 00:01:59+00:00,@Timothy58100352 @USABaseball @TeamUSA @WBSC @...,"Zanesville, OH","[[[-82.121534, 39.908312], [-81.976901, 39.908...",Place(_api=<tweepy.api.API object at 0x2b23538...
...,...,...,...,...,...,...,...
2863,1656,1421621097166901248,2021-07-31 23:57:36+00:00,This was team USA where you get team lebron fr...,"Nevada, USA","[[[-120.00574, 35.002086], [-114.039649, 35.00...",Place(_api=<tweepy.api.API object at 0x2b23538...
2864,2139,1421621301656043520,2021-07-31 23:58:25+00:00,@ASoftstar If that's the human standard...no w...,"Minneapolis, MN","[[[-93.329515, 44.889964], [-93.194578, 44.889...",Place(_api=<tweepy.api.API object at 0x2b23538...
2865,6779,1421621505885188098,2021-07-31 23:59:14+00:00,Way to rep Ohio 🙌🇺🇸🥇🥈🥉🎖 https://t.co/KmgqPJVahc,"Canal Fulton, OH","[[[-81.623096, 40.842646], [-81.536917, 40.842...",Place(_api=<tweepy.api.API object at 0x2b23538...
2866,1395,1421621510578581507,2021-07-31 23:59:15+00:00,I’m loving the guys calling the men’s @usavoll...,"Texas, USA","[[[-106.645646, 25.837092], [-93.508131, 25.83...",Place(_api=<tweepy.api.API object at 0x2b23538...


In [4]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
us_state_to_abbrev["Texas"]

'TX'

In [5]:
def append_states(date):
    cities_list = df['city_name']
    states_list = []
    for city in cities_list:
        city = city.split(', ')
        if len(city) <= 1:
            state = ""
        elif(city[1]=="USA"):
            state = city[0]
        else:
            state = city[1]
        if(state in us_state_to_abbrev.keys()):
            state = us_state_to_abbrev[state]
        states_list.append(state)
    df['state'] = states_list

In [6]:
for i in date_range:
    append_states(i.strftime('%Y-%m-%d'))
df.head(5)   

,id,time,text,city_name,coordinates,place,state
0,1419808468324790274,2021-07-26 23:54:52+00:00,I hadn’t seen you for awhile… I thought you we...,"Newport, KY","[[[-84.504476, 39.064988], [-84.4611, 39.06498...",Place(_api=<tweepy.api.API object at 0x2b23538...,KY
1,1419343586320019458,2021-07-25 17:07:35+00:00,There’s a bad joke here somewhere https://t.co...,"Inglewood, CA","[[[-118.378887, 33.9253169], [-118.3134177, 33...",Place(_api=<tweepy.api.API object at 0x2b23538...,CA
2,1419372964823408640,2021-07-25 19:04:20+00:00,The #Olympics seem to be starting out slowly f...,"Central City, KY","[[[-87.154448, 37.269115], [-87.0936138, 37.26...",Place(_api=<tweepy.api.API object at 0x2b23538...,KY
3,1425093660165738509,2021-08-10 13:56:20+00:00,https://t.co/u0G6OPzAav,"Gainesville, VA","[[[-77.675429, 38.763188], [-77.579294, 38.763...",Place(_api=<tweepy.api.API object at 0x2b23538...,VA
4,1419343311375151107,2021-07-25 17:06:30+00:00,@SpencerBoyd Olympics/Baseball,"Vicksburg, MI","[[[-85.583959, 42.089785], [-85.509529, 42.089...",Place(_api=<tweepy.api.API object at 0x2b23538...,MI


In [7]:
df_state = df.groupby('state', as_index=False).agg({"text": ' '.join, "time": "first"})    
df_state.head(5)

,state,text,time
0,,Already out here scouting for the Winter Olymp...,2021-07-29 00:43:50+00:00
1,AK,Buti na lng nanalo c Diaz sa Olympics nasapawa...,2021-07-27 08:00:28+00:00
2,AL,#Olympics Nikita Nagornyy #roc Summer Olympics...,2021-07-29 00:52:42+00:00
3,AR,Dang! I thought Naomi was about to get that br...,2021-07-26 02:37:46+00:00
4,AZ,Awesome news! https://t.co/hHeMU2XC0I This nee...,2021-08-04 00:11:38+00:00


In [8]:
df_state.to_dict()
df_state_dict = {}
state_range = df_state["state"].tolist()
new_state_range = []
for place in state_range:
    if place not in us_state_to_abbrev.values():
        continue
    new_state_range.append(place)
    df_state_dict[place] = df_state[(df_state['state']==place)]
df_state_dict["TX"]

,state,text,time
72,TX,They can’t hide all the news… you just have to...,2021-07-25 17:12:36+00:00


In [9]:
df_state_dict["TX"]["text"]

72    They can’t hide all the news… you just have to...
Name: text, dtype: object

In [10]:
#calculate topic freq
# document frequency: in how many files of the Brown corpus does each word appear
brown.fileids()
stopwords = nltk.corpus.stopwords.words("english")
stopwords += ["rt", "got", "thats", "would", "going", "u", 
               "get", "also", "one", "could", "said", "like",
               "via", "say", "amp", "many", "need", "day", "want", 
               "last", "people", "done", "two",
               "outbreak", "know", "upon", "may", "way",
               "saw", "gtgt", "olympics", "tokyo2020", "tokyo", "olympics2021", "usa"
               "olympics2020", "@olympics", "tokyoolympics", "olympic", "https"]

docfreq = { }

for brown_part in brown.fileids():
    fd = nltk.FreqDist(brown.words(fileids = brown_part))
    
    for word in fd.keys():
        word = word.lower()
        
        if word in stopwords:
            continue
        if not word.isalnum():
            continue
            
        if word in docfreq:
            docfreq[word] = docfreq[word] + 1
        else:
            docfreq[word] = 1

In [11]:
words_example = word_tokenize(df_state["text"][0])
#print(words_example)

In [12]:
# Use an NLTK FreqDist object to compute the frequency of each word in Frankenstein
def compute_freq_dict(sentence):
    freq_example = FreqDist()
    words = word_tokenize(sentence)
    for word in words:
        word = word.lower()

        if word in stopwords:
            continue
        if not word.isalnum():
            continue
        freq_example[word.lower()] += 1 
    return freq_example

In [13]:
df_state_dict["TX"]

,state,text,time
72,TX,They can’t hide all the news… you just have to...,2021-07-25 17:12:36+00:00


In [23]:
def append_word_freq(state):
    tweets_list = df_state_dict[state]['text']
    word_freq_dict_list = []
    for sentence in tweets_list:
        word_freq_dict = compute_freq_dict(sentence)
        word_freq_dict_list.append(word_freq_dict)    
    df_state_dict[state]["word_freq_dict"] = word_freq_dict_list

In [24]:
for state in new_state_range:
    append_word_freq(state)

In [25]:
df_state_dict["TX"]

,state,text,time,tf_idf,word_freq_dict
72,TX,They can’t hide all the news… you just have to...,2021-07-25 17:12:36+00:00,None,"{'hide': 2, 'look': 40, 'dstfamily': 3, 'que':..."


In [28]:
def compute_tf_idf(sentence):
    dfidf = FreqDist()
    num_brown_parts = len(list(brown.fileids()))
    words = word_tokenize(sentence)
    tf_dict = compute_freq_dict(sentence)
    for word in words:
        word = word.lower()

        if word in stopwords:
            continue
        if not word.isalnum():
            continue
        if word not in docfreq:
            continue
        df = docfreq[word]
        idf = math.log(num_brown_parts / df )
        dfidf[word] = tf_dict[word]*idf
def append_tf_idf_freq():
    tweets_list = df_state_dict[state]['text']
    word_tf_idf_list = []
    for sentence in tweets_list:
        word_tf_idf = compute_tf_idf(sentence)
        word_tf_idf_list.append(word_tf_idf)    
    df_state_dict[state]["tf_idf"] =  word_tf_idf_list

In [30]:
for state in new_state_range:
    append_word_freq(state)
df_state_dict["TX"].head(5)

,state,text,time,tf_idf,word_freq_dict
72,TX,They can’t hide all the news… you just have to...,2021-07-25 17:12:36+00:00,None,"{'hide': 2, 'look': 40, 'dstfamily': 3, 'que':..."


In [31]:
def plot_topic_mention(topic):
    freq_list = [freq_by_date[date.strftime('%Y-%m-%d')]['word_freq'][topic] for date in date_range]
    # import plotly.express as px
    fig = px.line(x=date_range, y=freq_list, title=topic+' mention frequnecy by time'
                 ,labels=dict(x="state", y= topic+" Mention Times", color="Place"))
    fig.show()  

In [20]:
hot_topics = ["covid", "cancel", "softball", "openingceremony", "skateboarding", "basketball", 
              "swimming", "simone", "gold", "marathon", "volleyball", "closing"]
for topic in hot_topics:
    plot_topic_mention(topic)

['', 'AK', 'AL', 'AR', 'AZ', 'Atlanta', 'Baltimore', 'Barrigada', 'Brooklyn', 'CA', 'CO', 'CT', 'Chicago', 'Cupertino', 'D.C.', 'DC', 'DE', 'Denver', 'El Paso', 'FL', 'Fort Worth', 'GA', 'HI', 'Houston', 'IA', 'ID', 'IL', 'IN', 'Indianapolis', 'Japan', 'KS', 'KY', 'LA', 'LLC', 'Las Vegas', 'Los Angeles', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'Manhattan', 'Memphis', 'Milwaukee', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'Nashville', 'OH', 'OK', 'OR', 'PA', 'PR', 'Paradise', 'Philadelphia', 'Portland', 'RI', 'SC', 'SD', 'San Diego', 'San Diego (UCSD)', 'San Francisco', 'Seattle', 'TN', 'TX', 'Tucson', 'Tumon', 'UT', 'VA', 'VT', 'Virgin Islands', 'WA', 'WI', 'WV', 'WY']


In [21]:
state_range = df_state["state"].tolist()
for place in state_range:
    if place not in us_state_to_abbrev.values():
        print(place)
        continue
    df_state_dict[place] = df_state[(df_state['state']==place)]


Atlanta
Baltimore
Barrigada
Brooklyn
Chicago
Cupertino
D.C.
Denver
El Paso
Fort Worth
Houston
Indianapolis
Japan
LLC
Las Vegas
Los Angeles
Manhattan
Memphis
Milwaukee
Nashville
Paradise
Philadelphia
Portland
San Diego
San Diego (UCSD)
San Francisco
Seattle
Tucson
Tumon
Virgin Islands


In [22]:
df_state_dict["TX"]["word_freq_dict"]

KeyError: 'word_freq_dict'

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.pylab import cm
# This lets us do slightly fancier LaTeX-based captions in plots
plt.rc('text', usetex=False)

In [ ]:
county_map = gpd.read_file('UScounties/UScounties.shp')

In [ ]:
county_map.plot()

In [ ]:
# Drop Alaska and Hawaii to just be the continental US
county_map = county_map[(county_map['STATE_NAME'] != 'Alaska') 
                          & (county_map['STATE_NAME'] != 'Hawaii')]

In [ ]:
# This is our fancy projection
projection = "+proj=laea +lat_0=30 +lon_0=-95"
county_map = county_map.to_crs(projection)

In [ ]:
fig, ax = plt.subplots(1, figsize=(11,8.5))
ax.axis('off')
county_map.plot(ax=ax)

In [ ]:
#append state and convert city to state in map
cities_list = county_map["STATE_NAME"]
states_list = []
for city in cities_list:
    state = us_state_to_abbrev[city]
    states_list.append(state)
county_map['state'] = states_list
county_map.head(5)

# Join the Data

In [ ]:
county_tweet_df = pd.merge(county_map, df_dict['2021-07-31'], on='state', how='inner')
county_tweet_df.head(5)